In [3]:
import nltk
from nltk.corpus import wordnet as reader
import nltk.corpus.reader.wordnet as wordnet
import sqlite3 as sql

In [5]:
DICTIONARY_PATH = 'data/output/dictionary.sqlite'

In [68]:
def initDictionary():
    import os

    # os.remove(DICTIONARY_PATH)

    with sql.connect(DICTIONARY_PATH) as conn:
        conn.execute("""
            CREATE TABLE word (
                word TEXT UNIQUE NOT NULL
            )
        """)
        conn.execute("""
            CREATE TABLE definition (
                wordId INTEGER,
                definition TEXT,
                pos TEXT,
                FOREIGN KEY(wordId) REFERENCES word(ROWID)
            )
        """)
        conn.execute("""
            CREATE TABLE example (
                example TEXT,
                definitionId INTEGER,
                FOREIGN KEY(definitionId) REFERENCES definition(ROWID)
            )
        """)

In [26]:
# need to do this because method suggestions don't work for `nltk.corpus.wordnet`
wn = wordnet.WordNetCorpusReader(reader.root, reader._omw_reader)

In [69]:
for i, word in enumerate(list(wn.words())):
    if i % 100 == 0:
        print(f'{i}: {word}')
    with sql.connect(DICTIONARY_PATH) as conn:
        wordCursor = conn.execute("""
            INSERT INTO word (word) VALUES(:word)
        """, {
            'word': word,
        })
        wordId = wordCursor.lastrowid

        for syn in wn.synsets(word):
            defCursor = conn.execute("""
                INSERT INTO definition (wordId, definition, pos)
                VALUES (:wordId, :definition, :pos)
            """, {
                'wordId': wordId,
                'definition': syn.definition(),
                'pos': syn.pos()
            })
            defId = defCursor.lastrowid

            for ex in syn.examples():
                if word not in ex:
                    continue
                conn.execute("""
                    INSERT INTO example (example, definitionId)
                    VALUES (:example, :definitionId)
                """, {
                    'example': ex,
                    'definitionId': defId
                })


0: .22-caliber
100: 2d
200: 79
300: abroach
400: acentric
500: acuate
600: adumbrative
700: after-hours
800: al_dente
900: all_right
1000: ambiguous
1100: anacoluthic
1200: angry
1300: anthropoidal
1400: aperient
1500: appreciable
1600: areal
1700: ascetical
1800: asymmetric
1900: auld
2000: avocado
2100: baffling
2200: baseborn
2300: begotten
2400: bewitched
2500: biloculate
2600: black-and-tan
2700: bloodstained
2800: bonzer
2900: brachiopodous
3000: bristlelike
3100: buff-coloured
3200: buxom
3300: camp
3400: cardiovascular
3500: catechistic
3600: cereal
3700: chelate
3800: christianly
3900: clashing
4000: clothesless
4100: coincidental
4200: commensurate
4300: concise
4400: consecrate
4500: contrite
4600: correspondent
4700: crack
4800: cross-modal
4900: curatorial
5000: cystic
5100: de_jure
5200: deep-seated
5300: demythologised
5400: despoiled
5500: dichromatic
5600: disarrayed
5700: dislogistic
5800: diversionary
5900: double-humped
6000: drum-like
6100: earsplitting
6200: egoce

In [64]:
for i, syn in enumerate(wn.synsets('good')):
    print(f'=============== {i} ===============')
    print(syn)
    print(syn.definition())
    print(syn.examples())
    print(syn.pos())
    print(syn.lemmas())
    # for lem in syn.lemmas():
    #     def iprint(text):
    #         print(f'\t{text}')
    #     iprint(lem)
    #     iprint(f'antonyms  : {lem.antonyms()}')
    #     iprint(f'causes    : {lem.causes()}')
    #     iprint(f'der_forms: {lem.derivationally_related_forms()}')
    #     iprint(f'in_usage_domains: {lem.in_usage_domains()}')
    #     iprint(f'attributes: {lem.attributes()}')

=============== 0 ===============
Synset('good.n.01')
benefit
['for your own good', "what's the good of worrying?"]
n
[Lemma('good.n.01.good')]
=============== 1 ===============
Synset('good.n.02')
moral excellence or admirableness
['there is much good to be found in people']
n
[Lemma('good.n.02.good'), Lemma('good.n.02.goodness')]
=============== 2 ===============
Synset('good.n.03')
that which is pleasing or valuable or useful
['weigh the good against the bad', 'among the highest goods of all are happiness and self-realization']
n
[Lemma('good.n.03.good'), Lemma('good.n.03.goodness')]
=============== 3 ===============
Synset('commodity.n.01')
articles of commerce
[]
n
[Lemma('commodity.n.01.commodity'), Lemma('commodity.n.01.trade_good'), Lemma('commodity.n.01.good')]
=============== 4 ===============
Synset('good.a.01')
having desirable or positive qualities especially those suitable for a thing specified
['good news from the hospital', 'a good report card', 'when she was good she w